In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} xgboost

In [68]:
import pandas as pd
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import xgboost
import random

## Pull the tumor gene expression data from kids first

In [27]:
gene_expression_data = pd.read_table("/sbgenomics/project-files/tumor-gene-expression-rsem-tpm-collapsed.tsv").transpose()

In [28]:
gene_expression_data.head()

,0,1,2,3,4,5,6,7,8,9,...,36708,36709,36710,36711,36712,36713,36714,36715,36716,36717
gene_id,MT-CO3,MT-CO2,MT-ATP6,MT-ND4,MT-CO1,MT-RNR2,MT-CYB,MT-ND2,MT-ND1,MT-ATP8,...,CT45A6,CT45A8,MIR718,RNU6-109P,RNU1-97P,TRAPPC2P9,TTTY17B,TRAPPC2P5,USP9YP9,RNU1-40P
BS_DERH44Z2,7458.52,8288.59,6606.88,6398.61,10211.13,2.04,3267.5,5137.5,4806.1,11633.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_CXGWN1W5,3883.93,3047.39,2996.46,4019.43,5824.88,2.51,1691.26,2157.91,3666.67,5305.92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_QS2NKZW3,4849.37,3168.87,3003.09,4011.17,6941.22,23.16,2616.85,3322.97,5265.57,3805.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_135GYQQD,1335.7,1378.69,932.8,1572.16,2357.34,0.48,687.46,846.49,965.47,1048.87,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
genes = gene_expression_data.iloc[0,:]

In [30]:
gene_expression_data = gene_expression_data.iloc[1:,:]
gene_expression_data.columns = genes

In [34]:
gene_expression_data.head()

gene_id,MT-CO3,MT-CO2,MT-ATP6,MT-ND4,MT-CO1,MT-RNR2,MT-CYB,MT-ND2,MT-ND1,MT-ATP8,...,CT45A6,CT45A8,MIR718,RNU6-109P,RNU1-97P,TRAPPC2P9,TTTY17B,TRAPPC2P5,USP9YP9,RNU1-40P
BS_DERH44Z2,7458.52,8288.59,6606.88,6398.61,10211.13,2.04,3267.5,5137.5,4806.1,11633.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_CXGWN1W5,3883.93,3047.39,2996.46,4019.43,5824.88,2.51,1691.26,2157.91,3666.67,5305.92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_QS2NKZW3,4849.37,3168.87,3003.09,4011.17,6941.22,23.16,2616.85,3322.97,5265.57,3805.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_135GYQQD,1335.7,1378.69,932.8,1572.16,2357.34,0.48,687.46,846.49,965.47,1048.87,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BS_HBMHZEGZ,2647.82,2342.92,2144.0,3229.79,4461.07,1.39,1638.03,2052.57,2867.3,3588.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
gene_expression_data = gene_expression_data.astype(float)

In [54]:
gene_expression_data.shape[0]

1284

In [55]:
cluster_assignments = random.choices(range(10), k = gene_expression_data.shape[0])

In [56]:
xtrain, xtest, ytrain, ytest = train_test_split(gene_expression_data, cluster_assignments, test_size=0.20)

In [69]:
clf = OneVsRestClassifier(xgboost.XGBClassifier())
clf.fit(xtest,ytest)

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:53:41] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:53:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:54:08] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:54:22] WARNING: /home

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
     

In [90]:
feature_importance = pd.DataFrame({'Cluster_{0}'.format(i):clf.estimators_[i].feature_importances_ for i in range(10)})

In [92]:
feature_importance.index = genes

In [97]:
feature_importance[feature_importance['Cluster_0']!=0.0]

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9
gene_id,,,,,,,,,,
FTH1,0.022210,0.0,0.0,0.0,0.005485,0.0,0.0,0.0,0.0,0.0
TMSB4X,0.018700,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
SPARCL1,0.000556,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
CSRP1,0.007165,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
BSG,0.005054,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
GLTPP1,0.007376,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
CYCSP43,0.012287,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
RNU6-21P,0.021203,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
